In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/xmanatee/Desktop/alyona/analytics


https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [ ]:
# copy token here
token = "b0eccfa8b4fe3dca107904cdcae88aee85291dfb37b141bf552fb8d625986dc3b0bbadf1384f4a9051abc"

In [ ]:
poll_group_id = "-172053584"

# VK API

In [ ]:
import vk
from vk.exceptions import VkAPIError

In [ ]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting wall data

### Creating data dir if it doesn't exist

In [ ]:
import os

In [ ]:
os.makedirs("../data", exist_ok=True)

### Requesting first 100

In [ ]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

group_wall_json = api.wall.get(**kwargs)

print("got {} items".format(len(group_wall_json["items"])))

### Requesting the rest

In [ ]:
import time

while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    print("Number of items now: {}".format(len(group_wall_json["items"])))
    time.sleep(3)

### Saving the result

In [ ]:
import json

In [ ]:
with open("data/group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [ ]:
import json

In [ ]:
with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [ ]:
def get_poll(wall_post):
    if "attachments" not in wall_post:
        return None
    for attachment in wall_post["attachments"]:
        if attachment["type"] != "poll":
            continue
        poll = attachment["poll"]
        return poll
    return None

In [ ]:
def vote_in_poll(api, poll):
    answer_id = poll["answers"][0]["id"]
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "answer_ids": [answer_id],
    }
    
    return api.polls.addVote(**kwargs)

In [ ]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    poll_filename = "data/poll_{}.json".format(poll["id"])

    if os.path.isfile(poll_filename):
        continue
    
    if poll["answer_id"] == 0:
        print("needs answering: https://vk.com/asurveys?w=poll-172053584_{}".format(poll["id"]))
        continue

    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
        print(vk_error)
        print(poll)
        time.sleep(1)
        continue
    with open(poll_filename, "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)

# Processing and squeezing data

In [ ]:
import json
import os

In [ ]:
# import json
# with open("data/poll_307034744.json", "r") as f:
#     poll = json.load(f)
# poll

### Building main mappings:

In [4]:
DATA_DIR = "../data/"

In [5]:
from common.main_mapping import *

In [6]:
poll_id_to_answer_id_to_user_ids, user_id_to_user = build_main_mapping(DATA_DIR)

In [7]:
with open(DATA_DIR + "user_id_to_user.json", "w") as f:
    json.dump(user_id_to_user, f)

In [8]:
with open(DATA_DIR + "poll_id_to_answer_id_to_user_ids.json", "w") as f:
    json.dump(poll_id_to_answer_id_to_user_ids, f)

Now we have variable 'polls' which is [POLL_ID \* ANSWER_ID \* USER] where ids are not real but just indexation.

# Now let's make some stats

In [ ]:
CHART_ITEM_NUMBER = 5

# YESNO

### Let's find all YesOrNo questions

In [ ]:
import json

with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [ ]:
# group_wall_json["items"][0]

In [ ]:
def clear_text(text):
    return ''.join(filter(str.isalpha, text.lower()))

In [ ]:
yes_no_polls = {}

for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
        
    answer_texts = list(map(lambda answer: clear_text(answer["text"]), poll["answers"]))
    if "да" in answer_texts and "нет" in answer_texts:
        
        yes_no_polls[poll["id"]] = {
            "id": poll["id"],
            "yes_id": poll["answers"][answer_texts.index("да")]["id"],
            "no_id": poll["answers"][answer_texts.index("нет")]["id"]
        }
        

In [ ]:
print("Number of yes/no questions: {}".format(len(yes_no_polls)))

In [ ]:
os.makedirs("datax/", exist_ok=True)

### Now let's find out who are YESNO people

In [ ]:
from collections import defaultdict

yes_cnt = defaultdict(int)
no_cnt = defaultdict(int)

for yes_no_poll in yes_no_polls.values():
    poll_filepath = "data/poll_{}.json".format(yes_no_poll["id"])

    try:
        with open(poll_filepath, "r") as f:
            poll = json.load(f)
    except FileNotFoundError as error:
        print(error)
        continue

    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        if answer["answer_id"] == yes_no_poll["yes_id"]:
            for user in answer["users"]["items"]:
                yes_cnt[user["id"]] += 1
        if answer["answer_id"] == yes_no_poll["no_id"]:
            for user in answer["users"]["items"]:
                no_cnt[user["id"]] += 1


In [ ]:
def get_max_cnt(cnt_dict):
    return list(filter(lambda key: cnt_dict[key] == max(cnt_dict.values()), cnt_dict))

In [ ]:
def get_max_cnts(cnt_dict):
    return list(map(lambda i: i[0], sorted(cnt_dict.items(), key=lambda i: i[1], reverse=True)[:CHART_ITEM_NUMBER]))

In [ ]:
def id_to_name(user_id):
    user = user_id_to_user[user_id]
    return user["first_name"] + " " + user["last_name"]

def ids_to_names(user_ids):
    return list(map(id_to_name, user_ids))

In [ ]:
stat = {
    "stat_id": "yes_fellas",
    "stat_icon": "icon-like",
    "stat_name": "ДАшки",
    "stat_description": "Тот самый разборчивый человек, что вечно отвечает ДА.",
    "user_ids": ids_to_names(get_max_cnts(yes_cnt))
}

with open("datax/stat_yes_fellas.json", "w") as f:
    json.dump(stat, f)

In [ ]:
stat = {
    "stat_id": "no_fellas",
    "stat_icon": "icon-dislike",
    "stat_name": "НЕТушки",
    "stat_description": "Тот самый другой разборчивый человек.",
    "user_ids": ids_to_names(get_max_cnts(no_cnt))
}

with open("datax/stat_no_fellas.json", "w") as f:
    json.dump(stat, f)

# Make 1x1 correlation

In [ ]:
import json

with open("data/poll_id_to_answer_id_to_user_ids.json", "r") as f:
    poll_id_to_answer_id_to_user_ids = json.load(f)

In [ ]:
user_id_to_user_id_to_ncommon_polls = {}
for poll_id in poll_id_to_answer_id_to_user_ids:
    poll_users = set()
    answer_id_to_user_ids = poll_id_to_answer_id_to_user_ids[poll_id]
    for answer_id in answer_id_to_user_ids:
        user_ids = answer_id_to_user_ids[answer_id]
        poll_users.update(user_ids)
    for user_id_1 in poll_users:
        if user_id_1 not in user_id_to_user_id_to_ncommon_polls:
            user_id_to_user_id_to_ncommon_polls[user_id_1] = {}
        for user_id_2 in poll_users:
            if user_id_2 == user_id_1:
                continue
            if user_id_2 not in user_id_to_user_id_to_ncommon_polls[user_id_1]:
                user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2] = 0
            user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2] += 1

In [ ]:
user_id_to_user_id_to_ncommon_answer = {}
for poll_id in poll_id_to_answer_id_to_user_ids:
    answer_id_to_user_ids = poll_id_to_answer_id_to_user_ids[poll_id]
    for answer_id in answer_id_to_user_ids:
        user_ids = answer_id_to_user_ids[answer_id]
        for user_id_1 in user_ids:
            if user_id_1 not in user_id_to_user_id_to_ncommon_answer:
                user_id_to_user_id_to_ncommon_answer[user_id_1] = {}
            for user_id_2 in user_ids:
                if user_id_2 == user_id_1:
                    continue
                if user_id_2 not in user_id_to_user_id_to_ncommon_answer[user_id_1]:
                    user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2] = 0
                user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2] += 1

In [ ]:
MIN_NUMBER_OF_COMMON_POLLS = 50

user_id_to_user_id_to_correlation = {}
for user_id_1 in user_id_to_user_id_to_ncommon_polls:
    for user_id_2 in user_id_to_user_id_to_ncommon_polls[user_id_1]:
        ncommon_polls = user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2]
        if user_id_1 not in user_id_to_user_id_to_ncommon_answer:
            continue
        if user_id_2 not in user_id_to_user_id_to_ncommon_answer[user_id_1]:
            continue
        ncommon_answer = user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2]
        if ncommon_polls > MIN_NUMBER_OF_COMMON_POLLS:
            if user_id_1 not in user_id_to_user_id_to_correlation:
                user_id_to_user_id_to_correlation[user_id_1] = {}
            if user_id_2 not in user_id_to_user_id_to_correlation[user_id_1]:
                user_id_to_user_id_to_correlation[user_id_1][user_id_2] = 1.0 * ncommon_answer / ncommon_polls

### Test:

In [ ]:
print("polls : {}".format(user_id_to_user_id_to_ncommon_polls[40048641][86824543]))

print("answers : {}".format(user_id_to_user_id_to_ncommon_answer[40048641][86824543]))

print("correlation : {}".format(user_id_to_user_id_to_correlation[40048641][86824543]))

In [ ]:
print("polls : {}".format(user_id_to_user_id_to_ncommon_polls[86824543][40048641]))

print("answers : {}".format(user_id_to_user_id_to_ncommon_answer[86824543][40048641]))

print("correlation : {}".format(user_id_to_user_id_to_correlation[86824543][40048641]))

In [ ]:
with open("data/user_id_to_user_id_to_correlation.json", "w") as f:
    json.dump(user_id_to_user_id_to_correlation, f)

### Finding best couple

In [ ]:
def gender(user_id):
    return user_id_to_user[user_id]["sex"]

In [ ]:
correlation_and_user_id_and_user_id = []

for user_id_1 in user_id_to_user_id_to_correlation:
    for user_id_2 in user_id_to_user_id_to_correlation[user_id_1]:
        if user_id_1 < user_id_2:
            continue
        # Enabling only hetero relations
        if gender(user_id_1) == gender(user_id_2):
            continue
        correlation = user_id_to_user_id_to_correlation[user_id_1][user_id_2]
        correlation_and_user_id_and_user_id.append((correlation, user_id_1, user_id_2))

In [ ]:
sorted(correlation_and_user_id_and_user_id, reverse=True)[:5]

In [ ]:
couples = list(
    filter(
        lambda i: i[0] > i[1],
        map(
            lambda i: i[1:],
            sorted(correlation_and_user_id_and_user_id, reverse=True))))[:CHART_ITEM_NUMBER]

In [ ]:
couple_names = list(map(lambda couple: id_to_name(couple[0]) + " + " + id_to_name(couple[1]), couples))

In [ ]:
couple_names

In [ ]:
stat = {
    "stat_id": "best_couples",
    "stat_icon": "icon-people",
    "stat_name": "Тыры-Пары",
    "stat_description": "Вот им есть о чем потрепаться.",
    "user_ids": couple_names,
}

with open("datax/stat_best_couples.json", "w") as f:
    json.dump(stat, f)

### Storing user stats

In [ ]:
import json

with open("data/user_id_to_user_id_to_correlation.json", "r") as f:
    user_id_to_user_id_to_correlation = json.load(f)

In [ ]:
a = 0
for user_id_1 in user_id_to_user_id_to_correlation:
    user_id_to_correlation = user_id_to_user_id_to_correlation[user_id_1]
    print("a: {} len: {}".format(user_id_1, len(user_id_to_correlation)))
    user_id_and_correlation = sorted(user_id_to_correlation.items(), key=lambda kv: kv[1], reverse=True)
    print(user_id_and_correlation[:CHART_ITEM_NUMBER])
    a += 1
    if a > 3:
        break